# Library

In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import pyexasol

In [484]:
# 소수점 2자리까지 표기
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

In [313]:
from tqdm import tqdm

# fold

In [3]:
raw_fold = 'D:/data/processing/urea/raw'
df_fold = 'D:/data/processing/urea/df'
an_fold = 'D:/data/processing/urea/an'
folds = {
    'raw_fold':raw_fold, 
    'df_fold':df_fold, 
    'an_fold':an_fold, 
}
folds

{'raw_fold': 'D:/data/processing/urea/raw',
 'df_fold': 'D:/data/processing/urea/df',
 'an_fold': 'D:/data/processing/urea/an'}

# server

In [4]:
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')

# load

## 등록제원 경유, 경유하이브리드

In [488]:
# 1m 21.8s
name = '경유_경유하이브리드_20231206'
file_name = f'{name}.csv'
ch_col = {
    'VIN':'차대번호', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소유무',
    'FRST_REG_YMD':'최초등록일자',
    'YRIDNW':'연식', 
    'FUEL_CD':'연료',
    'VHCTY_CD_x':'차종', 
    'PURPS_CD2':'용도', 
    'MNFCTR_NM':'제작사명', 
    'EGIN_TY':'엔진형식', 
    'TOTL_WGHT':'총중량', 
    'VHCNM':'차명', 
    'MANP_MNG_NO':'제원관리번호', 
    'EGIN_OTPT':'엔진출력', 
    'DSPLVL':'배기량', 
    
    # 'BSPL_STDG_CD':'법정동코드', 
    # 'VHCL_FBCTN_YMD':'제작일자', 
    # 'VHCL_MNG_NO':'차량관리번호', 
    # 'VHRNO':'자동차등록번호',
    # 'EXTGAS_INSP_VLD_YMD':'검사유효일',
    # 'VHCL_OWNR_CL_CD':'소유자구분',  
}

# cs_dgl = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, nrows=10)
# cs_dgl.columns

cs_dgl = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, usecols=ch_col.keys())
cs_dglr = cs_dgl.rename(columns=ch_col)
cs_dglr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11905778 entries, 0 to 11905777
Data columns (total 16 columns):
 #   Column    Dtype  
---  ------    -----  
 0   배출가스인증번호  object 
 1   배출가스등급    object 
 2   최초등록일자    object 
 3   용도        object 
 4   제원관리번호    object 
 5   차량말소유무    object 
 6   연식        int64  
 7   차종        object 
 8   차대번호      object 
 9   배기량       float64
 10  엔진출력      float64
 11  엔진형식      object 
 12  연료        object 
 13  제작사명      object 
 14  총중량       float64
 15  차명        object 
dtypes: float64(3), int64(1), object(12)
memory usage: 1.4+ GB


In [698]:
cs_dglr['차명'].isnull().sum()

0

In [490]:
cs_dglr['배출가스등급'].unique()

array(['A0505', 'A0503', 'A0504', nan], dtype=object)

In [491]:
## 배출가스등급 코드 변환
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
cs_dglr['배출가스등급'] = cs_dglr['배출가스등급'].replace(grd_dict)
cs_dglr['배출가스등급'].unique()

array(['5', '3', '4', nan], dtype=object)

In [492]:
# 4.0s
## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
cs_dglr['차종'] = cs_dglr['차종'].replace(cd_dict)
cs_dglr['차종'].unique()

array(['화물', '승용', '승합', '특수'], dtype=object)

In [493]:
## 용도 코드 변환
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
cs_dglr['용도'] = cs_dglr['용도'].replace(purps_dict)
cs_dglr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

In [494]:
# 7.9s
## 연료 코드 변환
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
cs_dglr['연료'] = cs_dglr['연료'].replace(fuel_dict)
cs_dglr['연료'].unique()

array(['경유', '경유 하이브리드'], dtype=object)

In [495]:
# ## 차종유형 코드 변환
# ty_dict = {
#     'A30C':'경형', 
#     'A30L':'대형', 
#     'A30M':'중형', 
#     'A30S':'소형',
# }
# cs_dglr['차종유형'] = cs_dglr['차종유형'].replace(ty_dict)

In [496]:
cs_dglr.shape

(11905778, 16)

## 등록정보

In [14]:
# # 1m 21.8s
# name = 'ods_ceg_car_mid'
# file_name = f'{name}.csv'
# # STD
# ch_col = {
#     'VIN':'차대번호', 
#     'EXHST_GAS_GRD_CD':'배출가스등급', 
#     'EXHST_GAS_CERT_NO':'배출가스인증번호',
#     'VHCL_ERSR_YN':'차량말소유무',
#     'MANP_MNG_NO':'제원관리번호', 
#     'YRIDNW':'연식', 
#     'VHCTY_CD':'차종', 
#     'PURPS_CD2':'용도', 
#     'FRST_REG_YMD':'최초등록일자',

#     # 'BSPL_STDG_CD':'법정동코드', 
#     # 'VHRNO':'자동차등록번호',
#     # 'VHCL_FBCTN_YMD':'제작일자', 
#     # 'VHCL_MNG_NO':'차량관리번호', 
#     # 'EXTGAS_INSP_VLD_YMD':'검사유효일',
#     # 'VHCL_OWNR_CL_CD':'소유자구분', 
# }

# # ODS
# ch_col = {
#     'VIN':'차대번호', 
#     'EMIS_GRD':'배출가스등급', 
#     'EMIS_CRTCNO':'배출가스인증번호',
#     'VHCL_ERSR_YN':'차량말소유무',
#     'SRCMNNO':'제원관리번호', 
#     'VHCL_YRIDNW':'연식', 
#     'VHCTY':'차종', 
#     'PURPS':'용도', 
#     'FST_REG_DE':'최초등록일자',

#     # 'BSPL_LEDO_CD':'법정동코드', 
#     # 'VHRNO':'자동차등록번호',
#     # 'VHCL_MNFCT_DE':'제작일자', 
#     # 'VHMNO':'차량관리번호', 
#     # 'EMIS_INSP_EFCT_DE':'검사유효일',
#     # 'OWNR_SE':'소유자구분', 
# }
# car = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, usecols=ch_col.keys())
# carr = car.rename(columns=ch_col)
# carr.info()

In [15]:
# carr.columns

In [16]:
# carr['배출가스등급'].unique()

In [17]:
# ## 배출가스등급 코드 변환
# grd_dict = {
#     'A0501':'1', 
#     'A0502':'2', 
#     'A0503':'3', 
#     'A0504':'4', 
#     'A0505':'5', 
#     'A05T2':'2',
#     'A05T3':'3', 
#     'A05T4':'4', 
#     'A05T5':'5', 
#     'A05X':'X', 
# }
# carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
# carr['배출가스등급'].unique()

In [18]:
# # 4.0s
# ## 차종 코드 변환
# cd_dict = {
#     'A31M':'이륜', 
#     'A31P':'승용', 
#     'A31S':'특수', 
#     'A31T':'화물', 
#     'A31V':'승합'
# }
# carr['차종'] = carr['차종'].replace(cd_dict)
# carr['차종'].unique()

In [19]:
# ## 용도 코드 변환
# purps_dict = {
#     'A08P':'개인용', 
#     'A08B':'영업용', 
#     'A08O':'관용',
# }
# carr['용도'] = carr['용도'].replace(purps_dict)
# carr['용도'].unique()

In [20]:
# carr.shape

In [21]:
# # 차대번호 중복 제거
# carr['최초등록일자'] = pd.to_numeric(carr['최초등록일자'], errors='coerce')
# carr['최초등록일자'].dtype
# # 19.4s
# carm = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
# carm.shape

## 제원정보

In [22]:
# # 1m 21.8s
# name = 'ods_ceg_car_src_mig'
# file_name = f'{name}.csv'
# # STD
# ch_col = {
#     'MANP_MNG_NO':'제원관리번호', 
#     'FUEL_CD':'연료',
#     'VHCTY_TY_CD2':'차종유형', 
#     'MNFCTR_NM':'제작사명', 
#     'VHCNM':'차명', 
#     'VHCL_FRM':'자동차형식', 
#     'EGIN_TY':'엔진형식', 
#     'TOTL_WGHT':'총중량',

#     # 'VHCTY_CL_CD':'차종분류',
#     # 'CRYNG_WGHT':'적재중량',
#     # 'DSPLVL':'배기량', 
#     # 'EGIN_OTPT':'엔진출력', 
# }

# # ODS
# ch_col = {
#     'SRCMNNO':'제원관리번호', 
#     'FUEL':'연료',
#     'VHCTY_TY':'차종유형', 
#     'MKR_NM':'제작사명', 
#     'VHCNM':'차명', 
#     'CAR_FRM':'자동차형식', 
#     'EGINTY':'엔진형식', 
#     'TOT_WGHT':'총중량',

#     # 'VHCTY_CL':'차종분류',
#     # 'CRYNG_WGHT':'적재중량',
#     # 'DSPLVL':'배기량', 
#     # 'EGINPWR':'엔진출력', 
# }
# src = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, usecols=ch_col.keys())
# srcr = src.rename(columns=ch_col)
# srcr.info()

In [23]:
# # 7.9s
# ## 연료 코드 변환
# fuel_dict = {
#     'A90GS':'휘발유', 
#     'A91DS':'경유',
#     'A92LP':'LPG(액화석유가스)', 
#     'A90GH':'휘발유 하이브리드', 
#     'A93EV':'전기', 
#     'A91DH':'경유 하이브리드', 
#     'A92CN':'CNG(압축천연가스)', 
#     'A93HD':'수소', 
#     'A92LH':'LPG 하이브리드', 
#     'A94OT':'기타연료', 
#     'A92CH':'CNG 하이브리드',
#     'A90AC':'알코올', 
#     'A93SH':'태양열', 
#     'A91KS':'등유', 
#     'A92LN':'LNG(액화천연가스)', 
#     'A90PH':'플러그인 하이브리드', 
# }
# srcr['연료'] = srcr['연료'].replace(fuel_dict)
# srcr['연료'].unique()

In [24]:
# ## 차종유형 코드 변환
# ty_dict = {
#     'A30C':'경형', 
#     'A30L':'대형', 
#     'A30M':'중형', 
#     'A30S':'소형',
# }
# srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)

## 등록&제원 병합

In [25]:
# srcr.shape, len(srcr['제원관리번호'].unique())

In [26]:
# carm.shape, len(carm['차대번호'].unique())

In [27]:
# cs = carm.merge(srcr, on='제원관리번호', how='left')
# cs.shape

## 경유차량, 말소제거, 중복차대 제거

In [497]:
cs = cs_dglr.copy()

In [498]:
cs.shape

(11905778, 16)

In [499]:
# 차대번호 중복 제거
cs['최초등록일자'] = pd.to_numeric(cs['최초등록일자'], errors='coerce')
cs['최초등록일자'].dtype
# 19.4s
cs = cs.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
cs.shape

(11902152, 16)

In [500]:
cs.groupby(['연료', '차량말소유무'], dropna=False)['차대번호'].count().reset_index()

,연료,차량말소유무,차대번호
0,경유,N,9533735
1,경유,Y,2316604
2,경유 하이브리드,N,48708
3,경유 하이브리드,Y,3105


In [501]:
# 5.0s
csm = cs[(cs['연료'] == '경유') & (cs['차량말소유무'] == 'N')].reset_index(drop=True)
csm.shape

(9533735, 16)

## 저감장치 부착이력

In [33]:
# 3.0s
att = wd.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
# STD
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분',
    # 'RDCDVC_KND_CD':'저감장치종류',
}

# # ODS
# att_ch_col = {
#     'VIN':'차대번호', 
#     'RDCDVC_SE':'저감장치구분',
#     # 'RDCDVC_KND':'저감장치종류',
# }
attr = att.rename(columns=att_ch_col)
print(attr.columns)
print('data load : STD_DLM_TB_ERP_ATT_HIS')

Index(['차대번호', '저감장치구분'], dtype='object')
data load : STD_DLM_TB_ERP_ATT_HIS


In [34]:
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['1종', '3종', '엔진개조', '2종', '엔진교체', '삼원촉매', '1종+SCR'], dtype=object)

In [35]:
attr.shape, len(attr['차대번호'].unique())

((1015941, 2), 1014369)

In [36]:
sorted(['1종', '3종', '엔진개조', '2종', '엔진교체', '삼원촉매', '1종+SCR'])

['1종', '1종+SCR', '2종', '3종', '삼원촉매', '엔진개조', '엔진교체']

In [37]:
attr.sort_values('저감장치구분').head()

,차대번호,저감장치구분
0,KL2UR52FDXP016197,1종
719912,KMHMJ81VP7U291509,1종
719913,KMFZSN7JP7U290144,1종
719914,KMFZCS7HP7U256498,1종
719917,KMFWVH7HP7U825459,1종


In [38]:
# 차대번호 중복 제거
attm = attr.sort_values('저감장치구분').drop_duplicates('차대번호').reset_index(drop=True)
attm.shape

(1014369, 2)

## 인증대장

In [39]:
name = '인증대장(대형)_SCR_적용차종(EURO 4, 5)'
file_name = f'{name}.xlsx'
l45 = pd.read_excel(os.path.join(raw_fold, file_name))
l45.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   제작사         42 non-null     object 
 1   차명          42 non-null     object 
 2   자동차형식       42 non-null     object 
 3   엔진형식        42 non-null     object 
 4   배기량         42 non-null     float64
 5   최대출력        43 non-null     object 
 6   Unnamed: 6  43 non-null     object 
 7   변속기         42 non-null     object 
 8   총중량         42 non-null     object 
 9   공차중량        42 non-null     object 
 10  제작사.1       42 non-null     object 
 11  제작국         42 non-null     object 
 12  인증번호        43 non-null     object 
 13  인증일자        43 non-null     object 
 14  비고          42 non-null     object 
dtypes: float64(1), object(14)
memory usage: 5.2+ KB


In [40]:
l45_col = [
    '제작사1',
    '차명',
    '자동차형식',
    '엔진형식',
    '배기량',
    '최대출력_PS',
    '최대출력_RPM',
    '변속기',
    '총중량',
    '공차중량',
    '제작사2',
    '제작국',
    '인증번호',
    '인증일자',
    '비고',
]
l45.columns = l45_col
l45.columns

Index(['제작사1', '차명', '자동차형식', '엔진형식', '배기량', '최대출력_PS', '최대출력_RPM', '변속기',
       '총중량', '공차중량', '제작사2', '제작국', '인증번호', '인증일자', '비고'],
      dtype='object')

In [41]:
l45.shape

(43, 15)

In [42]:
l45.head()

,제작사1,차명,자동차형식,엔진형식,배기량,최대출력_PS,최대출력_RPM,변속기,총중량,공차중량,제작사2,제작국,인증번호,인증일자,비고
0,NaN,NaN,NaN,NaN,NaN,PS,RPM,NaN,NaN,NaN,NaN,NaN,배출,배출,NaN
1,현대자동차,현대25.5톤덤프트럭,HD255-YDU0-DHECA,D6CE,12742.0,520,1700,자동12단,38730,13230,현대자동차,한국,AMY-HD-14-35,2010-07-19 00:00:00,유로5
2,타타대우,대우25톤장축카고트럭,BT9CC1,DV11,10964.0,440,1900,수동16단,38800,13670,타타대우상용차,한국,7MY-DC-23-70,2007-12-18 00:00:00,유로4
3,타타대우,대우5톤초장축카고트럭,BC4AH1,F4AE3681D,5880.0,251,2700,수동6단,11415,6285,타타대우상용차,한국,7MY-DC-23-72,2007-12-18 00:00:00,유로4
4,타타대우,대우5톤초장축카고트럭,BC4AT1,DL06,5890.0,270,2500,수동6단,11500,6370,타타대우상용차,한국,8MY-DC-24-12,2008-05-21 00:00:00,유로4


In [43]:
l45 = l45.iloc[1:].reset_index(drop=True)
l45.head()

,제작사1,차명,자동차형식,엔진형식,배기량,최대출력_PS,최대출력_RPM,변속기,총중량,공차중량,제작사2,제작국,인증번호,인증일자,비고
0,현대자동차,현대25.5톤덤프트럭,HD255-YDU0-DHECA,D6CE,12742.0,520,1700,자동12단,38730,13230,현대자동차,한국,AMY-HD-14-35,2010-07-19 00:00:00,유로5
1,타타대우,대우25톤장축카고트럭,BT9CC1,DV11,10964.0,440,1900,수동16단,38800,13670,타타대우상용차,한국,7MY-DC-23-70,2007-12-18 00:00:00,유로4
2,타타대우,대우5톤초장축카고트럭,BC4AH1,F4AE3681D,5880.0,251,2700,수동6단,11415,6285,타타대우상용차,한국,7MY-DC-23-72,2007-12-18 00:00:00,유로4
3,타타대우,대우5톤초장축카고트럭,BC4AT1,DL06,5890.0,270,2500,수동6단,11500,6370,타타대우상용차,한국,8MY-DC-24-12,2008-05-21 00:00:00,유로4
4,타타대우,대우트랙터,EX3TX2,F3BE3681B,12882.0,560,1900,자동12단,26020,8810,타타대우상용차,한국,8MY-DC-24-15,2008-06-30 00:00:00,유로4


In [44]:
name = '인증대장(대형)_SCR_적용차종(EURO 6)'
file_name = f'{name}.xlsx'
l6 = pd.read_excel(os.path.join(raw_fold, file_name))
l6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   No           21 non-null     int64 
 1   제작사          21 non-null     object
 2   인증번호         21 non-null     object
 3   인증일자         21 non-null     object
 4   차명           21 non-null     object
 5   차종           21 non-null     object
 6   차형식          21 non-null     object
 7   엔진형식         21 non-null     object
 8   배기량          21 non-null     int64 
 9   변속기          21 non-null     object
 10  최대출력         21 non-null     int64 
 11  Unnamed: 11  21 non-null     int64 
 12  총중량          21 non-null     object
 13  공차중량         21 non-null     object
 14  제작국          21 non-null     object
 15  사용연료         21 non-null     object
 16  배출기준         21 non-null     object
 17  유럽기준         21 non-null     object
 18  SCR유무        21 non-null     object
dtypes: int64(4), object(15)
memory 

In [45]:
name = '인증대장(소형)_SCR_적용차종(EURO 6)'
file_name = f'{name}.xlsx'
s6 = pd.read_excel(os.path.join(raw_fold, file_name))
s6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   일련번호          47 non-null     int64 
 1   제작사           47 non-null     object
 2   배출가스차종        47 non-null     object
 3   배출가스인증번호      47 non-null     object
 4   배출가스인증일       47 non-null     object
 5   차명            47 non-null     object
 6   자동차형식         47 non-null     object
 7   엔진형식          47 non-null     object
 8   배기량           47 non-null     int64 
 9   변속기           47 non-null     object
 10  최대출력PS        47 non-null     int64 
 11  최대출력RPM       47 non-null     int64 
 12  총중량           47 non-null     int64 
 13  공차중량          47 non-null     int64 
 14  제작국가          47 non-null     object
 15  사용연료          47 non-null     object
 16  배출가스기준        47 non-null     object
 17  유럽 기준         47 non-null     object
 18  SCR 유(Y)무(N)  47 non-null     object
dtypes: int64(6

In [46]:
# 확인된 사항
# l45, l6, s6 배출가스인증번호(인증번호) 중복 없음

In [47]:
len(set(l45['인증번호'].to_list()))

42

In [48]:
len(set(l45['인증번호'].to_list()) - set(l6['인증번호'].to_list()))

42

In [49]:
len(set(l6['인증번호'].to_list()))

21

In [50]:
len(set(l6['인증번호'].to_list()) - set(s6['배출가스인증번호'].to_list()))

21

In [51]:
len(set(l45['인증번호'].to_list()) - set(s6['배출가스인증번호'].to_list()))

42

## 제작사 구분

In [681]:
name = '엔진형식_제작사명_리스트(수정)'
file_name = f'{name}.xlsx'
ck = pd.read_excel(os.path.join(raw_fold, file_name))
ck.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8679 entries, 0 to 8678
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   엔진형식                 8679 non-null   object
 1   제작사명                 8679 non-null   object
 2   차대번호                 8679 non-null   int64 
 3   국내/외 구분(1:국내, 2:국외)  8679 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 271.3+ KB


In [682]:
ck.head()

,엔진형식,제작사명,차대번호,"국내/외 구분(1:국내, 2:국외)"
0,-,(주) 두성특장차,1,1
1,-,한국특장차(주),1,1
2,.D4GB,(주) 골드밴,1,1
3,0,SOFA차량,4,1
4,0,남도수입,1,2


In [683]:
ck = ck.rename(columns={'국내/외 구분(1:국내, 2:국외)':'구분'})
ck['구분'] = ck['구분'].replace({1:'국내', 2:'국외'})
ck.head()

,엔진형식,제작사명,차대번호,구분
0,-,(주) 두성특장차,1,국내
1,-,한국특장차(주),1,국내
2,.D4GB,(주) 골드밴,1,국내
3,0,SOFA차량,4,국내
4,0,남도수입,1,국외


In [684]:
ck = ck.drop(['차대번호'], axis=1)
ck.head()

,엔진형식,제작사명,구분
0,-,(주) 두성특장차,국내
1,-,한국특장차(주),국내
2,.D4GB,(주) 골드밴,국내
3,0,SOFA차량,국내
4,0,남도수입,국외


In [685]:
ck.shape, ck.drop_duplicates(['엔진형식', '제작사명']).shape

((8679, 3), (8679, 3))

## 220630_기준_SCR_부착_차량_통계자료_수정

In [57]:
scr = pd.read_excel(os.path.join(raw_fold, '220630_기준_SCR_부착_차량_통계자료_수정.xlsx'))
scr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   인증번호      501 non-null    object
 1   차종        500 non-null    object
 2   원본차종      500 non-null    object
 3   제조사       500 non-null    object
 4   차명        351 non-null    object
 5   통계자료존재유무  501 non-null    object
dtypes: object(6)
memory usage: 23.6+ KB


In [58]:
scr = scr[['인증번호', '차종', '제조사', '차명', '통계자료존재유무']]
scr.shape

(501, 5)

In [59]:
scr.isnull().sum()

인증번호          0
차종            1
제조사           1
차명          150
통계자료존재유무      0
dtype: int64

In [60]:
scr.shape, len(scr['인증번호'].unique())

((501, 5), 501)

## SCR 장착 확인 차량

In [428]:
scr_ck = pd.read_excel(os.path.join(raw_fold, '경유차량_SCR_통계_20231216.xlsx'))
scr_ck.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65321 entries, 0 to 65320
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   배출가스인증번호                   27771 non-null  object 
 1   제원관리번호                     65321 non-null  object 
 2   제작사명                       65321 non-null  object 
 3   연식                         65321 non-null  int64  
 4   차종                         65321 non-null  object 
 5   엔진형식                       65321 non-null  object 
 6   배기량                        65321 non-null  int64  
 7   엔진출력                       65320 non-null  float64
 8   차명                         65321 non-null  object 
 9   유럽기준                       17601 non-null  object 
 10  저감장치구분                     487 non-null    object 
 11  통계자료존재유무                   23301 non-null  object 
 12  차량대수                       65321 non-null  int64  
 13  SCR 장착여부 확인 (1:장착, 2:미장착)  64834 non-null  flo

In [429]:
scr_ck.head()

,배출가스인증번호,제원관리번호,제작사명,연식,차종,엔진형식,배기량,엔진출력,차명,유럽기준,저감장치구분,통계자료존재유무,차량대수,"SCR 장착여부 확인 (1:장착, 2:미장착)"
0,NaN,0003412h000061221,자기인증면제차량,2015,승용,20953,1598,100.0,MERCEDES-BENZ C200,NaN,NaN,NaN,1,1.0
1,NaN,2GY20000300003320,전라남도소방본부,2019,화물,542972,15930,700.0,ZIEGLER내폭화학소방차,NaN,NaN,NaN,1,1.0
2,BMY-MK-12-8,00820003200111313,메르세데스벤츠코리아(주),2015,승용,642826,2987,258.0,ML350 BlueTEC 4Matic,NaN,NaN,NaN,2,1.0
3,BMY-MK-12-8,00820003200121314,메르세데스벤츠코리아(주),2015,승용,642826,2987,258.0,ML350 BlueTEC 4Matic,NaN,NaN,NaN,718,1.0
4,NaN,00031105000111319,자기인증면제차량,2016,승용,642826,2987,258.0,MERCEDES-BENZ GLE350d,NaN,NaN,NaN,1,1.0


In [430]:
scr_ck['SCR 장착여부 확인 (1:장착, 2:미장착)'].value_counts(dropna=False)

1.0    60041
2.0     4793
NaN      487
Name: SCR 장착여부 확인 (1:장착, 2:미장착), dtype: int64

# 전처리
- 컬럼명 수정
- 필수 컬럼만 추출
- 등록 + 인증대장 + 저감
- 등록 + 인증대장 + 저감 + SCR 통계자료(차종, 제조사, 차명 우선)

## 인증대장 병합

In [65]:
l45.columns.to_list()

['제작사1',
 '차명',
 '자동차형식',
 '엔진형식',
 '배기량',
 '최대출력_PS',
 '최대출력_RPM',
 '변속기',
 '총중량',
 '공차중량',
 '제작사2',
 '제작국',
 '인증번호',
 '인증일자',
 '비고']

In [66]:
l45.head()

,제작사1,차명,자동차형식,엔진형식,배기량,최대출력_PS,최대출력_RPM,변속기,총중량,공차중량,제작사2,제작국,인증번호,인증일자,비고
0,현대자동차,현대25.5톤덤프트럭,HD255-YDU0-DHECA,D6CE,12742.0,520,1700,자동12단,38730,13230,현대자동차,한국,AMY-HD-14-35,2010-07-19 00:00:00,유로5
1,타타대우,대우25톤장축카고트럭,BT9CC1,DV11,10964.0,440,1900,수동16단,38800,13670,타타대우상용차,한국,7MY-DC-23-70,2007-12-18 00:00:00,유로4
2,타타대우,대우5톤초장축카고트럭,BC4AH1,F4AE3681D,5880.0,251,2700,수동6단,11415,6285,타타대우상용차,한국,7MY-DC-23-72,2007-12-18 00:00:00,유로4
3,타타대우,대우5톤초장축카고트럭,BC4AT1,DL06,5890.0,270,2500,수동6단,11500,6370,타타대우상용차,한국,8MY-DC-24-12,2008-05-21 00:00:00,유로4
4,타타대우,대우트랙터,EX3TX2,F3BE3681B,12882.0,560,1900,자동12단,26020,8810,타타대우상용차,한국,8MY-DC-24-15,2008-06-30 00:00:00,유로4


In [67]:
l45m = l45[[
    '인증번호',
    '제작사2',
    '차명',
    '제작국',
    '비고'

    # '제작사1',
    # '자동차형식',
    # '엔진형식',
    # '배기량',
    # '최대출력_PS',
    # '최대출력_RPM',
    # '변속기',
    # '총중량',
    # '공차중량',
    # '인증일자',
]]
l45m.head()

,인증번호,제작사2,차명,제작국,비고
0,AMY-HD-14-35,현대자동차,현대25.5톤덤프트럭,한국,유로5
1,7MY-DC-23-70,타타대우상용차,대우25톤장축카고트럭,한국,유로4
2,7MY-DC-23-72,타타대우상용차,대우5톤초장축카고트럭,한국,유로4
3,8MY-DC-24-12,타타대우상용차,대우5톤초장축카고트럭,한국,유로4
4,8MY-DC-24-15,타타대우상용차,대우트랙터,한국,유로4


In [68]:
l45m = l45m.rename(columns={'인증번호':'배출가스인증번호', '제작사2':'제작사', '제작국가':'제작사', '비고':'유럽기준'})
l45m['SCR유무'] = 'Y'
l45m.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,AMY-HD-14-35,현대자동차,현대25.5톤덤프트럭,한국,유로5,Y
1,7MY-DC-23-70,타타대우상용차,대우25톤장축카고트럭,한국,유로4,Y
2,7MY-DC-23-72,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
3,8MY-DC-24-12,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
4,8MY-DC-24-15,타타대우상용차,대우트랙터,한국,유로4,Y


In [69]:
l6.head()

,No,제작사,인증번호,인증일자,차명,차종,차형식,엔진형식,배기량,변속기,최대출력,Unnamed: 11,총중량,공차중량,제작국,사용연료,배출기준,유럽기준,SCR유무
0,6,자일대우버스(주),LMY-DB-14-105,2020-12-31,BX212,승용(초대형),BX212M-GPLBZC441L,OM470LA.6-6,10677,자동화수동변속기,456,1600,16035,13110,한국,경유(Diesel),2017년 10월 기준,유로6,Y
1,8,타타대우상용차(주),LMY-DC-14-87,2020-11-03,LD트럭,화물(대형),FE5BC1,F4AFE411C,4485,수동6단,206,2500,9910,4715,한국,경유(Diesel),2017년 10월 기준,유로6,Y
2,12,타타대우상용차(주),KMY-DC-14-82,2019-11-26,프리마14톤장축카고트럭,화물(초대형),JL8A82,DX12P,11051,자동12단,460,1900,25800(25260),11670(11130),한국,경유(Diesel),2017년 10월 기준,유로6,Y
3,17,(주)우진산전,GMY-WJ-14-02,2016-01-06,우진산전 굴절버스,승용(대형),KM9ZGCZE,ISB6.7E6,6690,없음,280,2100,25995,19495,한국,경유-하이브리드(D-Hybrid),2014년 1월 기준,유로6,Y
4,19,타타대우상용차(주),GMY-DC-14-52,2016-11-07,노부스5톤극초장축플러스8.6카고트럭,화물(대형),DC8CT7,DL06P,5890,수동 9단,280,2500,12500(12710),7370(7580),한국,경유(Diesel),2014년 1월 기준,유로6,Y


In [70]:
l6.columns.to_list()

['No',
 '제작사',
 '인증번호',
 '인증일자',
 '차명',
 '차종',
 '차형식',
 '엔진형식',
 '배기량',
 '변속기',
 '최대출력',
 'Unnamed: 11',
 '총중량',
 '공차중량',
 '제작국',
 '사용연료',
 '배출기준',
 '유럽기준',
 'SCR유무']

In [71]:
l6m = l6[[
    '인증번호',
    '제작사',
    '차명',
    '제작국',
    '유럽기준',
    'SCR유무',
    
    # 'No',
    # '인증일자',
    # '차종',
    # '차형식',
    # '엔진형식',
    # '배기량',
    # '변속기',
    # '최대출력',
    # 'Unnamed: 11',
    # '총중량',
    # '공차중량',
    # '사용연료',
    # '배출기준',
]]
l6m.head()

,인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,LMY-DB-14-105,자일대우버스(주),BX212,한국,유로6,Y
1,LMY-DC-14-87,타타대우상용차(주),LD트럭,한국,유로6,Y
2,KMY-DC-14-82,타타대우상용차(주),프리마14톤장축카고트럭,한국,유로6,Y
3,GMY-WJ-14-02,(주)우진산전,우진산전 굴절버스,한국,유로6,Y
4,GMY-DC-14-52,타타대우상용차(주),노부스5톤극초장축플러스8.6카고트럭,한국,유로6,Y


In [72]:
l6m = l6m.rename(columns={'인증번호':'배출가스인증번호'})
l6m.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,LMY-DB-14-105,자일대우버스(주),BX212,한국,유로6,Y
1,LMY-DC-14-87,타타대우상용차(주),LD트럭,한국,유로6,Y
2,KMY-DC-14-82,타타대우상용차(주),프리마14톤장축카고트럭,한국,유로6,Y
3,GMY-WJ-14-02,(주)우진산전,우진산전 굴절버스,한국,유로6,Y
4,GMY-DC-14-52,타타대우상용차(주),노부스5톤극초장축플러스8.6카고트럭,한국,유로6,Y


In [73]:
s6.head()

,일련번호,제작사,배출가스차종,배출가스인증번호,배출가스인증일,차명,자동차형식,엔진형식,배기량,변속기,최대출력PS,최대출력RPM,총중량,공차중량,제작국가,사용연료,배출가스기준,유럽 기준,SCR 유(Y)무(N)
0,27,현대자동차(주),승용(중형),FMY-HD-14-38,2015-07-29,그랜드스타렉스 2.5디젤 12인(4WD),TQ-25D12-E6-4W5A,D4CB,2497,자동5단,175,3600,3395,2505,한국,경유(Diesel),2014년 1월 기준,유로6,Y
1,29,현대자동차(주),승용(소형),HMY-HD-14-61,2017-12-15,G80 2.2디젤 2WD,DH-22D-H6-2W8A,D4HC,2199,자동8단,202,3800,2435,2025,한국,경유(Diesel),2017년 10월 기준,유로6,Y
2,31,현대자동차(주),승용(소형),HMY-HD-14-63,2017-12-15,G70 2.2 디젤 2WD,IK-22D-E6C-2W8A,D4HC,2199,자동8단,202,3800,2140,1765,한국,경유(Diesel),2017년 10월 기준,유로6,Y
3,32,현대자동차(주),승용(소형),JMY-HD-14-15,2018-02-02,싼타페 2.0디젤 2WD,TM-20D7-H6-2W8A,D4HA,1995,자동8단,186,4000,2485,1910,한국,경유(Diesel),2017년 10월 기준,유로6,Y
4,33,현대자동차(주),승용(소형),HMY-HD-14-62,2017-12-15,G70 2.2 디젤 AWD,IK-22D-E6C-AW8A,D4HC,2199,자동8단,202,3800,2215,1840,한국,경유(Diesel),2017년 10월 기준,유로6,Y


In [74]:
s6.columns.to_list()

['일련번호',
 '제작사',
 '배출가스차종',
 '배출가스인증번호',
 '배출가스인증일',
 '차명',
 '자동차형식',
 '엔진형식',
 '배기량',
 '변속기',
 '최대출력PS',
 '최대출력RPM',
 '총중량',
 '공차중량',
 '제작국가',
 '사용연료',
 '배출가스기준',
 '유럽 기준',
 'SCR 유(Y)무(N)']

In [75]:
s6m = s6[[
    '배출가스인증번호',
    '제작사',
    '차명',
    '제작국가',
    '유럽 기준',
    'SCR 유(Y)무(N)'

    # '일련번호',
    # '배출가스차종',
    # '배출가스인증일',
    # '자동차형식',
    # '엔진형식',
    # '배기량',
    # '변속기',
    # '최대출력PS',
    # '최대출력RPM',
    # '총중량',
    # '공차중량',
    # '사용연료',
    # '배출가스기준',
]]
s6m.head()

,배출가스인증번호,제작사,차명,제작국가,유럽 기준,SCR 유(Y)무(N)
0,FMY-HD-14-38,현대자동차(주),그랜드스타렉스 2.5디젤 12인(4WD),한국,유로6,Y
1,HMY-HD-14-61,현대자동차(주),G80 2.2디젤 2WD,한국,유로6,Y
2,HMY-HD-14-63,현대자동차(주),G70 2.2 디젤 2WD,한국,유로6,Y
3,JMY-HD-14-15,현대자동차(주),싼타페 2.0디젤 2WD,한국,유로6,Y
4,HMY-HD-14-62,현대자동차(주),G70 2.2 디젤 AWD,한국,유로6,Y


In [76]:
s6m = s6m.rename(columns={'SCR 유(Y)무(N)':'SCR유무', '제작국가':'제작국', '유럽 기준':'유럽기준'})
s6m.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,FMY-HD-14-38,현대자동차(주),그랜드스타렉스 2.5디젤 12인(4WD),한국,유로6,Y
1,HMY-HD-14-61,현대자동차(주),G80 2.2디젤 2WD,한국,유로6,Y
2,HMY-HD-14-63,현대자동차(주),G70 2.2 디젤 2WD,한국,유로6,Y
3,JMY-HD-14-15,현대자동차(주),싼타페 2.0디젤 2WD,한국,유로6,Y
4,HMY-HD-14-62,현대자동차(주),G70 2.2 디젤 AWD,한국,유로6,Y


In [77]:
l45m.shape[0] + l6m.shape[0] + s6m.shape[0]

110

In [78]:
cert = pd.concat([l45m, l6m, s6m], ignore_index=True)
cert.shape

(110, 6)

In [79]:
cert[cert['차명'].str.contains(r'\n')]

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
22,AMY-SK-14-02,스카니아,스카니아트랙터\n(견인형),스웨덴,유로5,Y
23,AMY-SK-14-03,스카니아,스카니아카고트럭\n(견인형),스웨덴,유로5,Y
24,CMY-SK-14-01,스카니아,스카니아카고트럭\n(견인형),스웨덴,유로5,Y


In [80]:
cert['차명'] = cert['차명'].str.replace('\n', '')
cert.loc[22:24, :]

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
22,AMY-SK-14-02,스카니아,스카니아트랙터(견인형),스웨덴,유로5,Y
23,AMY-SK-14-03,스카니아,스카니아카고트럭(견인형),스웨덴,유로5,Y
24,CMY-SK-14-01,스카니아,스카니아카고트럭(견인형),스웨덴,유로5,Y


In [81]:
cert['제작사'].value_counts(dropna=False)

현대자동차(주)      40
타타대우상용차       11
대우버스          10
타타대우상용차(주)     8
현대자동차㈜         7
현대자동차          6
볼보트럭코퍼레이션      5
Daimler AG     5
자일대우버스(주)      5
스카니아           3
만              3
이베코            3
기아(주)          2
진화영맨           1
(주)우진산전        1
Name: 제작사, dtype: int64

In [82]:
cert['제작국'].value_counts(dropna=False)

한국      90
스웨덴      8
독일       8
이탈리아     3
중국       1
Name: 제작국, dtype: int64

In [83]:
cert['유럽기준'].value_counts(dropna=False)

유로6    68
유로5    33
유로4     9
Name: 유럽기준, dtype: int64

In [84]:
cert['SCR유무'].value_counts(dropna=False)

Y    110
Name: SCR유무, dtype: int64

In [85]:
cert.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,AMY-HD-14-35,현대자동차,현대25.5톤덤프트럭,한국,유로5,Y
1,7MY-DC-23-70,타타대우상용차,대우25톤장축카고트럭,한국,유로4,Y
2,7MY-DC-23-72,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
3,8MY-DC-24-12,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
4,8MY-DC-24-15,타타대우상용차,대우트랙터,한국,유로4,Y


In [86]:
cert.isnull().sum()

배출가스인증번호    0
제작사         0
차명          0
제작국         0
유럽기준        0
SCR유무       0
dtype: int64

### [출력] 인증대장

In [87]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20231216'

In [88]:
# # 0.0s
# cert.to_csv(os.path.join(raw_fold, f'인증대장_{today_date}.csv'), index=False)
# print(os.path.join(raw_fold, f'인증대장_{today_date}.csv'))

### [로드] 인증대장 종합

In [699]:
cert = pd.read_csv(os.path.join(raw_fold, '인증대장_20231214.csv'))
cert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   배출가스인증번호  110 non-null    object
 1   제작사       110 non-null    object
 2   차명        110 non-null    object
 3   제작국       110 non-null    object
 4   유럽기준      110 non-null    object
 5   SCR유무     110 non-null    object
dtypes: object(6)
memory usage: 5.3+ KB


## 등록 + 인증

In [722]:
csm.shape

(9533735, 16)

In [723]:
cc = csm.copy()

In [742]:
csm[['제작사명', '엔진형식', '차명']].isnull().sum()

제작사명     592
엔진형식    1311
차명         0
dtype: int64

In [724]:
cert.columns

Index(['배출가스인증번호', '제작사', '차명', '제작국', '유럽기준', 'SCR유무'], dtype='object')

In [725]:
cert.shape

(110, 6)

In [726]:
# 1m 6.8s
for one, two, three, four, five, six in cert.values:
    cc.loc[cc['배출가스인증번호'] == one, ['제작사명', '차명', '제작국', '유럽기준', 'SCR유무']] = [two, three, four, five, six]

In [727]:
cc.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '용도', '제원관리번호', '차량말소유무', '연식', '차종',
       '차대번호', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준',
       'SCR유무'],
      dtype='object')

In [728]:
cc[['제작사명', '차명', '제작국', '유럽기준', 'SCR유무']].isnull().sum()

제작사명         592
차명             0
제작국      8473081
유럽기준     8473081
SCR유무    8473081
dtype: int64

## 등록 + 인증 + 저감

In [729]:
# 13.8s
cca = cc.merge(attm, on='차대번호', how='left')
cca.shape

(9533735, 20)

In [730]:
cca.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '용도', '제원관리번호', '차량말소유무', '연식', '차종',
       '차대번호', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준',
       'SCR유무', '저감장치구분'],
      dtype='object')

In [731]:
cca[['엔진형식', '제작사명']].isnull().sum()

엔진형식    1311
제작사명     592
dtype: int64

In [732]:
cca[(cca['엔진형식'].isnull()) & (cca['제작사명'].isnull())].shape

(168, 20)

In [735]:
cca.loc[(cca['엔진형식'].isnull()) & (cca['제작사명'].isnull()), '차명'].isnull().sum()

0

In [736]:
cca.loc[(cca['엔진형식'].isnull()) & (cca['제작사명'].isnull()), '차명'].unique()

array(['이스스', '타다노대형렉카트럭', '바나곤 GL', '코로나웨곤', '시발 G.M.C', '스키니아구난차',
       '제설차', '캔워스트랙타', '벤즈300TD보통승합', '고려식트럭', '도요다크라운로얄살롱', '이동방송차',
       '인타트레이러', '렌드로바', '미쓰버시렉카추럭', '비리몬트트렉타', '장비운반트럭', '후소디젤담프',
       'G M C', '중앙식덤푸트럭', '캔위즈 트렉타', '찌프 웨곤', '렉카트럭', '하이드로켄 세이트레일러',
       '후소15톤카고트럭', '카이자카고트럭', '후소카고', '캔위즈트렉타', '인타담프', '후소탱크로리', '히노카고',
       '히노CBU담프', '히노탱크로리', '후소카고트럭', '후소11톤카고', '후소', '국일GMC담프', '춘천식화물',
       '도요다랜드크루사', '후소(렉카)', '인타트렉타', '랜드로바', '인타덤프', '맥크트랙타', '인타',
       '고려식 G M C', '인타5톤레카차', '진디추레라샤시', '트렉타', '시발G.M.C', '닛산트랙타',
       '서울보데식추럭', '콘테이너샷시트레일러', '콘테이너 트레라', '지엠시', '후소 트레라', '히노 트레일러',
       '삼양식트럭', '히다치샷시', '대전식 덤프', '고려식화물', '인타담프추럭', '인타트렉타견인용',
       '서울보레식G.M.C', '이스스트랙타', '한국화물식 추럭', '성암식GMC', '국제식G.M.C', '시발GMC',
       '시발식 G M C', '지엠씨', 'GMC덤프', 'G M C 담뿌'], dtype=object)

### [출력] 제작사명, 엔진형식
- 국내외 구분 위해

In [737]:
ex = cca.groupby(['엔진형식', '제작사명'], dropna=False)['차대번호'].count().reset_index()
ex.shape

(8637, 3)

In [738]:
ex.head()

,엔진형식,제작사명,차대번호
0,-,(주) 두성특장차,1
1,-,한국특장차(주),1
2,.D4GB,(주) 골드밴,1
3,0,SOFA차량,4
4,0,남도수입,1


In [741]:
ex[(ex['엔진형식'].isnull()) & (ex['제작사명'].isnull())]

,엔진형식,제작사명,차대번호
8636,NaN,NaN,168


In [279]:
# exc = ex[['엔진형식', '제작사명']].merge(ck, on=['엔진형식', '제작사명'], how='left')
# exc1 = exc[exc['구분'].isnull()].reset_index(drop=True)
# exc1.shape

In [280]:
# today_date = datetime.today().strftime('%Y%m%d')
# today_date

In [281]:
# ex.to_excel(os.path.join(an_fold, f'엔진형식_제작사명_리스트_{today_date}.xlsx'), index=False)

In [282]:
# exc1.to_excel(os.path.join(an_fold, f'추가_엔진형식_제작사명_리스트_{today_date}.xlsx'), index=False)

## 제작사 국내외 구분 설정

In [709]:
cca.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '용도', '제원관리번호', '차량말소유무', '연식', '차종',
       '차대번호', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준',
       'SCR유무', '저감장치구분'],
      dtype='object')

In [710]:
# 제작사 구분
ck.columns

Index(['엔진형식', '제작사명', '구분'], dtype='object')

In [711]:
len(set(cca['엔진형식'].values) - set(ck['엔진형식'].values))

113

In [712]:
set(cca['엔진형식'].values) - set(ck['엔진형식'].values)

{'01744200032',
 '01781200884',
 '01781200916',
 '01831103592',
 '02-81-11-77160',
 '02701202291',
 '02791164595',
 '02791167174',
 '02791167182',
 '02791167971',
 '02801168480',
 '02801171957',
 '03017806811310333',
 '03701216281',
 '03701270607',
 '03721290085',
 '03741217014',
 '03741221799',
 '03741229497',
 '03751251614',
 '03801173800',
 '04-72-12-00656',
 '06711201574',
 '07-78-12-00934',
 '07834301513',
 '07851301678',
 '09684120002',
 '09761200515',
 '10781152968',
 '10801125134',
 '10804205108',
 '10864103435',
 '10PCI',
 '3306',
 '3306C',
 '3406C',
 '3406E',
 '356.956',
 '357 913',
 '357.912',
 '357.913',
 '364 982',
 '364.982',
 '366.948',
 '366.963',
 '401 979',
 '413B',
 '4D31',
 '59A/3',
 '601.941',
 '601940',
 '617.952',
 '6BD1-T',
 '6D16',
 '6HH1',
 '7.8L',
 '8210.42K',
 '8210.42R',
 '821022',
 '8DC11',
 '8DC4',
 '8DC8-2A',
 '8DC81A',
 '8VF',
 '8dc81a',
 '9.0DM2U28977',
 '92A/3',
 'BF8L513',
 'C8.3-210',
 'D-3.9L',
 'D2866LF03',
 'D41 8.3',
 'D421.85',
 'D7',
 'D706LT'

In [713]:
merge_col = ['엔진형식', '제작사명']

In [714]:
cca.loc[cca['엔진형식'] == '01744200032', merge_col]

,엔진형식,제작사명
9533183,01744200032,NaN


In [715]:
set(ck['엔진형식'].values) - set(cca['엔진형식'].values)

set()

In [718]:
ck.loc[ck['엔진형식'] == '01744200032', merge_col]

,엔진형식,제작사명


In [719]:
len(set(csm['제작사명'].values) - set(ck['제작사명'].values))

9

In [720]:
set(csm['제작사명'].values) - set(ck['제작사명'].values)

{'(주)진도',
 '(주)케이원모터스',
 '(주)한원모터스',
 nan,
 '대원기계',
 '모터팍',
 '세일특장',
 '해동정공',
 '혼다코리아(주)'}

In [697]:
len(set(ck['제작사명'].values) - set(cca['제작사명'].values))

28

In [666]:
cca['엔진형식'] = cca['엔진형식'].astype('str')
ck['엔진형식'] = ck['엔진형식'].astype('str')

In [667]:
ccac = cca.merge(ck, on=['제작사명', '엔진형식'], how='left')
ccac.shape

(9533735, 21)

In [668]:
ccac['구분'].isnull().sum()

429555

In [516]:
ccac.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '용도', '제원관리번호', '차량말소유무', '연식', '차종',
       '차대번호', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준',
       'SCR유무', '저감장치구분', '구분'],
      dtype='object')

## 등록 + 인증 + 저감 + 제작국 + SCR 통계

In [517]:
scr.head()

,인증번호,차종,제조사,차명,통계자료존재유무
0,AMY-HD-14-35,화물,현대,"트라고/엑시언트 (L엔진, D6CE)",Y
1,EMY-HD-14-45,승합,현대,"유니버스 / 엑시언트 (L엔진, D6CG)",Y
2,EMY-HD-14-48,승합,현대,"유니버스,뉴슈퍼에어로시티 / 엑시언트 (H엔진, D6HC)",Y
3,EMY-HD-14-51,승합,현대,"현대그린시티, 이-에어로타운 / 메가트럭, 메가트럭(와이드캡), 파비스 (G엔진, ...",Y
4,EMY-HD-14-53,승합,현대,"뉴카운티 / 마이티 (F엔진, D4GB)",Y


In [518]:
scr.shape

(501, 5)

In [519]:
scr['차명'].isnull().sum()

150

In [520]:
ccac[ccac['배출가스인증번호'].str.contains(' ', regex=False, na=False)]

,배출가스인증번호,배출가스등급,최초등록일자,용도,제원관리번호,차량말소유무,연식,차종,차대번호,배기량,...,엔진형식,연료,제작사명,총중량,차명,제작국,유럽기준,SCR유무,저감장치구분,구분


In [521]:
scr[scr['인증번호'].str.contains(' ', regex=False, na=False)]

,인증번호,차종,제조사,차명,통계자료존재유무


In [522]:
ccac['배출가스인증번호'].dtype

dtype('O')

In [523]:
scr['인증번호'].dtype

dtype('O')

In [524]:
ccac.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '용도', '제원관리번호', '차량말소유무', '연식', '차종',
       '차대번호', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준',
       'SCR유무', '저감장치구분', '구분'],
      dtype='object')

In [525]:
ccac[['차종', '제작사명', '차명']].head()

,차종,제작사명,차명
0,승합,현대자동차(주),스타리아 (STARIA)
1,화물,쌍용자동차(주),렉스턴스포츠 쿨멘 칸
2,승합,현대자동차(주),뉴-카운티
3,승합,자기인증면제차량,이-카운티
4,승합,현대자동차(주),뉴-카운티 어린이운송차


In [526]:
scr.head()

,인증번호,차종,제조사,차명,통계자료존재유무
0,AMY-HD-14-35,화물,현대,"트라고/엑시언트 (L엔진, D6CE)",Y
1,EMY-HD-14-45,승합,현대,"유니버스 / 엑시언트 (L엔진, D6CG)",Y
2,EMY-HD-14-48,승합,현대,"유니버스,뉴슈퍼에어로시티 / 엑시언트 (H엔진, D6HC)",Y
3,EMY-HD-14-51,승합,현대,"현대그린시티, 이-에어로타운 / 메가트럭, 메가트럭(와이드캡), 파비스 (G엔진, ...",Y
4,EMY-HD-14-53,승합,현대,"뉴카운티 / 마이티 (F엔진, D4GB)",Y


In [527]:
pr = ccac.copy()

In [528]:
scr['차명'].isnull().sum()

150

In [529]:
not pd.isna(scr.loc[scr['차명'].isnull(), '차명'].values[0])

False

In [530]:
# 5m 17.2s
for one, two, three, four, five in scr.values: # 배출가스인증번호, 차종, 제조사, 차명, 통계자료존재유무
    if one == 'FMY-DW-14-34': # FMY-DW-14-34의 값(차종, 제조사, 차명)이 비어있음
        pr.loc[pr['배출가스인증번호'] == one, '통계자료존재유무'] = five
    elif not pd.isna(four): # SCR통계에 차명이 존재하는 경우
        pr.loc[pr['배출가스인증번호'] == one, ['차종', '제작사명', '차명', '통계자료존재유무']] = [two, three, four, five]
    else: # SCR통계에 차명이 존재하지 않는 경우
        pr.loc[pr['배출가스인증번호'] == one, ['차종', '제작사명', '통계자료존재유무']] = [two, three, five]

In [531]:
pr['차명'].isnull().sum()

0

In [532]:
pr.shape

(9533735, 22)

In [533]:
pr['통계자료존재유무'].isnull().sum()

7379999

In [534]:
pr[['차종', '제작사명', '차명', '구분']].isnull().sum()

차종           0
제작사명       592
차명           0
구분      429555
dtype: int64

In [535]:
pr.head()

,배출가스인증번호,배출가스등급,최초등록일자,용도,제원관리번호,차량말소유무,연식,차종,차대번호,배기량,...,연료,제작사명,총중량,차명,제작국,유럽기준,SCR유무,저감장치구분,구분,통계자료존재유무
0,NaN,NaN,20231129.00,개인용,A0810012801972223,N,2024,승합,KMJYB371BRU160978,2199.00,...,경유,현대자동차(주),3035.00,스타리아 (STARIA),NaN,NaN,NaN,NaN,국내,NaN
1,NaN,NaN,20231129.00,개인용,A0510002302113123,N,2024,화물,KPADM4AE1RP216080,2157.00,...,경유,쌍용자동차(주),3005.00,렉스턴스포츠 쿨멘 칸,NaN,NaN,NaN,NaN,국내,NaN
2,NaN,NaN,20231129.00,영업용,A0810005611852223,N,2024,승합,KMJHG17GPRC005566,3933.00,...,경유,현대자동차(주),6195.00,뉴-카운티,NaN,NaN,NaN,NaN,국내,NaN
3,NaN,NaN,20231129.00,개인용,00034427000122223,N,2012,승합,KMJHG17HPCC011410,3933.00,...,경유,자기인증면제차량,5965.00,이-카운티,NaN,NaN,NaN,NaN,국내,NaN
4,NaN,NaN,20231129.00,영업용,A0810005612022223,N,2024,승합,KMJHK17GPRC005512,3933.00,...,경유,현대자동차(주),6430.00,뉴-카운티 어린이운송차,NaN,NaN,NaN,NaN,국내,NaN


In [536]:
pr.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '용도', '제원관리번호', '차량말소유무', '연식', '차종',
       '차대번호', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준',
       'SCR유무', '저감장치구분', '구분', '통계자료존재유무'],
      dtype='object')

In [537]:
pr.shape

(9533735, 22)

### [출력] 전처리 완료 파일

In [538]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20231217_AM_123124'

In [539]:
# 1m 40.6s
pr.to_csv(os.path.join(raw_fold, f'등록_인증_저감_국내외구분_SCR통계참고_{today_date_dtl}.csv'), index=False)
print(os.path.join(raw_fold, f'등록_인증_저감_국내외구분_SCR통계참고_{today_date_dtl}.csv'))

D:/data/processing/urea/raw\등록_인증_저감_국내외구분_SCR통계참고_20231217_AM_123124.csv


### [로드] 전처리 완료 파일

In [93]:
# # 31.0s
# pr = pd.read_csv(os.path.join(raw_fold, '등록_인증_저감_국내외구분_SCR통계참고_20231217_AM_123124.csv'))
# pr.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17400\403363107.py:1: DtypeWarning: Columns (3,15,16,17,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  pr = pd.read_csv(os.path.join(raw_fold, '등록_인증_저감_국내외구분_SCR통계참고_20231214_PM_094352.csv'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9533735 entries, 0 to 9533734
Data columns (total 21 columns):
 #   Column    Dtype  
---  ------    -----  
 0   배출가스인증번호  object 
 1   배출가스등급    float64
 2   최초등록일자    float64
 3   제원관리번호    object 
 4   차량말소유무    object 
 5   연식        int64  
 6   차종        object 
 7   차대번호      object 
 8   배기량       float64
 9   엔진출력      float64
 10  엔진형식      object 
 11  연료        object 
 12  제작사명      object 
 13  총중량       float64
 14  차명        object 
 15  제작국       object 
 16  유럽기준      object 
 17  SCR유무     object 
 18  저감장치구분    object 
 19  구분        object 
 20  통계자료존재유무  object 
dtypes: float64(5), int64(1), object(15)
memory usage: 1.5+ GB


In [540]:
pr['차명'].isnull().sum()

0

## 저공해조치 차량과 SCR차량 구분

In [314]:
# 저공해조치차량
    # 저감장치구분 : 1종+SCR인 차량
# SCR차량(EURO6)
    # SCR유무 : Y
    # 국내 : 2017년 이상
    # 국외 : 2015년 이상

In [541]:
mnt_idx = pr[pr['저감장치구분'] == '1종+SCR'].index
len(mnt_idx)

2372

In [542]:
mnt = pr[pr['저감장치구분'] == '1종+SCR'].reset_index(drop=True)
mnt.shape

(2372, 22)

### [출력] 저공해조치 차량

In [543]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20231217_AM_123306'

In [544]:
# # 1.7s
# mnt.to_excel(os.path.join(df_fold, f'경유_저공해조치차량_{today_date_dtl}.xlsx'), index=False)

In [545]:
pr['구분'].isnull().sum()

429555

In [546]:
euro_idx = pr[(pr['저감장치구분'] != '1종+SCR') & ( ((pr['구분'] == '국내') & (pr['연식'] >= 2017)) | ((pr['구분'] == '국외') & (pr['연식'] >= 2015)) | (pr['SCR유무'] == 'Y') )].index
len(euro_idx)

4433293

In [547]:
euro = pr[(pr['저감장치구분'] != '1종+SCR') & ( ((pr['구분'] == '국내') & (pr['연식'] >= 2017)) | ((pr['구분'] == '국외') & (pr['연식'] >= 2015)) | (pr['SCR유무'] == 'Y') )].reset_index(drop=True)
euro.shape

(4433293, 22)

In [548]:
pr.shape

(9533735, 22)

In [549]:
total_idx = set(pr.index)
rest_idx = total_idx - set(mnt_idx) - set(euro_idx)
len(rest_idx)

5098070

In [550]:
rest = pr.loc[list(rest_idx)].reset_index(drop=True)
rest.shape

(5098070, 22)

In [551]:
pr.shape[0] == len(mnt_idx) + len(euro_idx) + len(rest_idx)

True

In [552]:
div = 50 * 10**4
for n in range(euro.shape[0] // div + 1):
    print(n+1, n*div, (n+1) *div)

1 0 500000
2 500000 1000000
3 1000000 1500000
4 1500000 2000000
5 2000000 2500000
6 2500000 3000000
7 3000000 3500000
8 3500000 4000000
9 4000000 4500000


In [553]:
# # 
# div = 50 * 10**4
# for n in range(euro.shape[0] // div + 1):
#     # print(n*div, (n+1) *div)
#     euro.iloc[n*div:(n+1)*div].to_excel(os.path.join(df_fold, f'EURO차량현황{n+1}_{today_date}.xlsx'), index=False)

# 분석

## SCR 장착 차량 현황

In [554]:
mnt.shape[0] + euro.shape[0]

4435665

In [555]:
me = pd.concat([mnt, euro], ignore_index=False).reset_index(drop=True)
me.shape

(4435665, 22)

In [556]:
me.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '용도', '제원관리번호', '차량말소유무', '연식', '차종',
       '차대번호', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준',
       'SCR유무', '저감장치구분', '구분', '통계자료존재유무'],
      dtype='object')

In [557]:
# 3.6s
# megrp1 = me.groupby(['배출가스인증번호', '제작사명', '연식', '차종', '차명', '통계자료존재유무'], dropna=False)['차대번호'].count().reset_index()
megrp1 = me.groupby(['배출가스인증번호', '제원관리번호', '제작사명', '연식', '차종', '엔진형식', '배기량', '엔진출력', '차명', '유럽기준', '저감장치구분', '통계자료존재유무'], dropna=False)['차대번호'].count().reset_index()
megrp1 = megrp1.rename(columns={'차대번호':'차량대수'})
megrp1

,배출가스인증번호,제원관리번호,제작사명,연식,차종,엔진형식,배기량,엔진출력,차명,유럽기준,저감장치구분,통계자료존재유무,차량대수
0,1MY-HD-52,A0810005400273304,현대자동차(주),2004,화물,D6CA,12920.00,440.00,현대슈퍼트럭,NaN,1종+SCR,NaN,3
1,2MY-DW-04,A0310006500023303,타타대우상용차(주),2003,화물,ISM,10824.00,415.00,대우25톤카고트럭,NaN,1종+SCR,NaN,1
2,2MY-DW-04,A0310006500023303,타타대우상용차(주),2004,화물,ISM,10824.00,415.00,대우25톤카고트럭,NaN,1종+SCR,NaN,2
3,2MY-DW-04,A0310006500073303,타타대우상용차(주),2004,화물,ISM,10824.00,415.00,대우25톤카고트럭,NaN,1종+SCR,NaN,27
4,2MY-DW-04,A0310006701013306,타타대우상용차(주),2007,화물,ISM,10824.00,385.00,대우11.5톤장축카고트럭,NaN,1종+SCR,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65316,NaN,EFW20000200004223,(주)영남캠핑카,2023,특수,F1CFL4117,2998.00,180.00,영남캠핑카,NaN,NaN,NaN,1
65317,NaN,EG810000100003323,토마토특장,2024,화물,F3GGE611A,11120.00,480.00,토마토12.5톤윙바디트럭,NaN,NaN,NaN,1
65318,NaN,EGA20000100003323,이노특장,2023,화물,DC13164,12742.00,450.00,이노특장16톤상승윙바디,NaN,NaN,NaN,1
65319,NaN,EGA20000300003323,이노특장,2024,화물,DC13173,12742.00,560.00,이노특장23톤상승윙바디,NaN,NaN,NaN,1


In [558]:
megrp1['차량대수'].sum()

4435665

In [559]:
megrp1.isnull().sum()

배출가스인증번호    37550
제원관리번호          0
제작사명            0
연식              0
차종              0
엔진형식            0
배기량             0
엔진출력            1
차명              0
유럽기준        47720
저감장치구분      64834
통계자료존재유무    42020
차량대수            0
dtype: int64

In [560]:
megrp1.loc[megrp1['저감장치구분'].notnull(), '차량대수'].sum()

2372

In [561]:
megrp1.loc[(megrp1['유럽기준'].notnull()) & (megrp1['통계자료존재유무'].isnull()), '차량대수'].sum()

66790

In [562]:
megrp1.loc[(megrp1['유럽기준'].isnull()) & (megrp1['통계자료존재유무'].notnull()), '차량대수'].sum()

1114766

In [563]:
megrp1.loc[(megrp1['유럽기준'].notnull()) | (megrp1['통계자료존재유무'].notnull()), '차량대수'].sum()

2175420

In [564]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20231217_AM_123328'

### [출력] 경유차량 통계

In [565]:
# # 19.0s
# megrp1.to_excel(os.path.join(an_fold, f'경유차량_통계_{today_date_dtl}.xlsx'), index=False)

## SCR 장착 확인 차량 정보 추가

In [566]:
scr_ck.head()

,배출가스인증번호,제원관리번호,제작사명,연식,차종,엔진형식,배기량,엔진출력,차명,유럽기준,저감장치구분,통계자료존재유무,차량대수,"SCR 장착여부 확인 (1:장착, 2:미장착)"
0,NaN,0003412h000061221,자기인증면제차량,2015,승용,20953,1598,100.00,MERCEDES-BENZ C200,NaN,NaN,NaN,1,1.00
1,NaN,2GY20000300003320,전라남도소방본부,2019,화물,542972,15930,700.00,ZIEGLER내폭화학소방차,NaN,NaN,NaN,1,1.00
2,BMY-MK-12-8,00820003200111313,메르세데스벤츠코리아(주),2015,승용,642826,2987,258.00,ML350 BlueTEC 4Matic,NaN,NaN,NaN,2,1.00
3,BMY-MK-12-8,00820003200121314,메르세데스벤츠코리아(주),2015,승용,642826,2987,258.00,ML350 BlueTEC 4Matic,NaN,NaN,NaN,718,1.00
4,NaN,00031105000111319,자기인증면제차량,2016,승용,642826,2987,258.00,MERCEDES-BENZ GLE350d,NaN,NaN,NaN,1,1.00


In [567]:
scr_ckm = scr_ck.reset_index(drop=True).copy()
scr_ckm['확인'] = 1

In [568]:
me.head()

,배출가스인증번호,배출가스등급,최초등록일자,용도,제원관리번호,차량말소유무,연식,차종,차대번호,배기량,...,연료,제작사명,총중량,차명,제작국,유럽기준,SCR유무,저감장치구분,구분,통계자료존재유무
0,4MY-HD-34,5,20201223.00,영업용,A0810005301663304,N,2005,화물,KMFDA18TP5C003803,12344.00,...,경유,현대자동차(주),22710.00,뉴파워트럭,NaN,NaN,NaN,1종+SCR,국내,NaN
1,NaN,5,20191118.00,영업용,00034416000083319,N,2006,화물,KMFDA18TP6C011717,12344.00,...,경유,자기인증면제차량,22605.00,뉴파워트럭,NaN,NaN,NaN,1종+SCR,국내,NaN
2,4MY-HD-55,5,20171027.00,영업용,A0810005302353305,N,2006,화물,KMFDA18BP6C007799,11149.00,...,경유,현대자동차(주),16400.00,뉴 파워트럭,NaN,NaN,NaN,1종+SCR,국내,NaN
3,4MY-HD-55,5,20151028.00,개인용,A0810005301343304,N,2005,화물,KMFDA18BP5C002336,11149.00,...,경유,현대자동차(주),19185.00,뉴 파워트럭,NaN,NaN,NaN,1종+SCR,국내,NaN
4,4MY-HD-55,5,20141126.00,영업용,A0810005301343304,N,2005,화물,KMFDA18BP5C002335,11149.00,...,경유,현대자동차(주),19185.00,뉴 파워트럭,NaN,NaN,NaN,1종+SCR,국내,NaN


### 엔진형식 빈칸 제거

In [569]:
me.loc[me['엔진형식'].str.contains(' '), '엔진형식'].head()

5612      651 921
16534     651 921
49311     654 920
121842    654 920
130562    651 950
Name: 엔진형식, dtype: object

In [570]:
me['엔진형식'] = me['엔진형식'].str.replace(' ', '')
me.loc[[5612, 16534, 49311], '엔진형식']

5612     651921
16534    651921
49311    654920
Name: 엔진형식, dtype: object

In [571]:
me['엔진형식'].dtype

dtype('O')

In [572]:
scr_ckm['엔진형식'].dtype

dtype('O')

#### 오류 수정
- scr_ck : 20953 -> '020953'
- 엔진형식 str 타입으로 수정

In [573]:
me.loc[me['엔진형식'] == '020953', '엔진형식']

801702    020953
Name: 엔진형식, dtype: object

In [574]:
me['엔진형식'] = me['엔진형식'].astype('str')

In [575]:
scr_ckm.loc[scr_ckm['엔진형식'] == 20953, '엔진형식']

0    20953
Name: 엔진형식, dtype: object

In [576]:
scr_ckm['엔진형식'] = scr_ckm['엔진형식'].astype('str')
scr_ckm['엔진형식'] = scr_ckm['엔진형식'].str.replace('20953', '020953')

In [577]:
scr_ckm.loc[scr_ckm['엔진형식'] == 20953, '엔진형식']

Series([], Name: 엔진형식, dtype: object)

In [578]:
scr_ckm.loc[scr_ckm['엔진형식'] == '020953', '엔진형식']

0    020953
Name: 엔진형식, dtype: object

### 제원관리번호 타입 변환
- str

In [579]:
me['제원관리번호'] = me['제원관리번호'].astype('str')

In [580]:
megrp1.shape

(65321, 13)

In [581]:
scr_ckm.shape

(65321, 15)

### 차명 오류 수정
- 제원관리번호 : 02420006800231217
- me : Passat GT 2.0 TDI 4Motion
- scr_ckm : Passat GT 2.0 TDI -> Passat GT 2.0 TDI 4Motion

In [582]:
me.loc[me['제원관리번호'] == '02420006800231217', '차명'].unique()

array(['Passat GT 2.0 TDI 4Motion'], dtype=object)

In [583]:
scr_ckm.loc[scr_ckm['제원관리번호'] == '02420006800231217', '차명'].unique()

array(['Passat GT 2.0 TDI'], dtype=object)

In [584]:
scr_ckm.loc[scr_ckm['제원관리번호'] == '02420006800231217', '차명'] = 'Passat GT 2.0 TDI 4Motion'

In [585]:
scr_ckm.loc[scr_ckm['제원관리번호'] == '02420006800231217', '차명'].unique()

array(['Passat GT 2.0 TDI 4Motion'], dtype=object)

In [586]:
scr_ckm.groupby('유럽기준', dropna=False)['차량대수'].sum(), me['유럽기준'].value_counts(dropna=False)

(유럽기준
 유로4       1640
 유로5      23953
 유로6    1035061
 NaN    3375011
 Name: 차량대수, dtype: int64,
 NaN    3375011
 유로6    1035061
 유로5      23953
 유로4       1640
 Name: 유럽기준, dtype: int64)

In [587]:
scr_ckm.groupby('저감장치구분', dropna=False)['차량대수'].sum(), me['저감장치구분'].value_counts(dropna=False)

(저감장치구분
 1종+SCR       2372
 NaN       4433293
 Name: 차량대수, dtype: int64,
 NaN       4433293
 1종+SCR       2372
 Name: 저감장치구분, dtype: int64)

In [588]:
scr_ckm.groupby('통계자료존재유무', dropna=False)['차량대수'].sum(), me['통계자료존재유무'].value_counts(dropna=False)

(통계자료존재유무
 Y      2108630
 NaN    2327035
 Name: 차량대수, dtype: int64,
 NaN    2327035
 Y      2108630
 Name: 통계자료존재유무, dtype: int64)

In [589]:
me.shape

(4435665, 22)

In [590]:
me.dtypes

배출가스인증번호     object
배출가스등급       object
최초등록일자      float64
용도           object
제원관리번호       object
차량말소유무       object
연식            int64
차종           object
차대번호         object
배기량         float64
엔진출력        float64
엔진형식         object
연료           object
제작사명         object
총중량         float64
차명           object
제작국          object
유럽기준         object
SCR유무        object
저감장치구분       object
구분           object
통계자료존재유무     object
dtype: object

In [591]:
scr_ckm.dtypes

배출가스인증번호                      object
제원관리번호                        object
제작사명                          object
연식                             int64
차종                            object
엔진형식                          object
배기량                            int64
엔진출력                         float64
차명                            object
유럽기준                          object
저감장치구분                        object
통계자료존재유무                      object
차량대수                           int64
SCR 장착여부 확인 (1:장착, 2:미장착)    float64
확인                             int64
dtype: object

### 타입 변환

In [592]:
merge_col = ['배출가스인증번호', '제원관리번호', '제작사명', '연식', '차종', '엔진형식', '배기량', '엔진출력', '차명', '유럽기준', '저감장치구분', '통계자료존재유무']

In [593]:
me[merge_col].isnull().sum()

배출가스인증번호     598814
제원관리번호            0
제작사명              0
연식                0
차종                0
엔진형식              0
배기량               0
엔진출력              1
차명                0
유럽기준        3375011
저감장치구분      4433293
통계자료존재유무    2327035
dtype: int64

In [594]:
# '배출가스인증번호', '제원관리번호', '제작사명', '연식', '차종', '엔진형식', '배기량', '엔진출력', '차명', '유럽기준', '저감장치구분', '통계자료존재유무'
me['배출가스인증번호'] = me['배출가스인증번호'].astype('str')
# me['제원관리번호'] = me['제원관리번호'].astype('str') # 위에서 진행함
me['제작사명'] = me['제작사명'].astype('str')
me['연식'] = me['연식'].astype('int')
me['차종'] = me['차종'].astype('str')
# me['엔진형식'] = me['엔진형식'].astype('str') # 위에서 진행함
me['배기량'] = me['배기량'].astype('int')
me['엔진출력'] = pd.to_numeric(me['엔진출력'], errors='coerce')
me['차명'] = me['차명'].astype('str')

In [595]:
scr_ckm[merge_col].isnull().sum()

배출가스인증번호    37550
제원관리번호          0
제작사명            0
연식              0
차종              0
엔진형식            0
배기량             0
엔진출력            1
차명              0
유럽기준        47720
저감장치구분      64834
통계자료존재유무    42020
dtype: int64

In [596]:
# '배출가스인증번호', '제원관리번호', '제작사명', '연식', '차종', '엔진형식', '배기량', '엔진출력', '차명', '유럽기준', '저감장치구분', '통계자료존재유무'
scr_ckm['배출가스인증번호'] = scr_ckm['배출가스인증번호'].astype('str')
# scr_ckm['제원관리번호'] = scr_ckm['제원관리번호'].astype('str') # 위에서 진행함
scr_ckm['제작사명'] = scr_ckm['제작사명'].astype('str')
scr_ckm['연식'] = scr_ckm['연식'].astype('int')
scr_ckm['차종'] = scr_ckm['차종'].astype('str')
# scr_ckm['엔진형식'] = scr_ckm['엔진형식'].astype('str') # 위에서 진행함
scr_ckm['배기량'] = scr_ckm['배기량'].astype('int')
scr_ckm['엔진출력'] = pd.to_numeric(scr_ckm['엔진출력'], errors='coerce')
scr_ckm['차명'] = scr_ckm['차명'].astype('str')

In [597]:
# 6.0s
merge_col = ['배출가스인증번호', '제원관리번호', '제작사명', '연식', '차종', '엔진형식', '배기량', '엔진출력', '차명']
# '배출가스인증번호', '제원관리번호', '제작사명', '연식', '차종', '엔진형식', '배기량', '엔진출력', '차명', '유럽기준', '저감장치구분', '통계자료존재유무'
me_pr = me.merge(scr_ckm[[
    '배출가스인증번호',
    '제원관리번호',
    '제작사명',
    '연식',
    '차종',
    '엔진형식',
    '배기량',
    '엔진출력',
    '차명',
    # '유럽기준',
    # '저감장치구분',
    # '통계자료존재유무',
    # '차량대수',
    'SCR 장착여부 확인 (1:장착, 2:미장착)',
    '확인',
    ]], on=merge_col, how='left')
me_pr.shape

(4435665, 24)

In [598]:
me_pr['확인'].value_counts(dropna=False)

1    4435665
Name: 확인, dtype: int64

In [599]:
scr_ck.groupby('SCR 장착여부 확인 (1:장착, 2:미장착)', dropna=False)['차량대수'].sum()

SCR 장착여부 확인 (1:장착, 2:미장착)
1.00    3204754
2.00    1228539
NaN        2372
Name: 차량대수, dtype: int64

In [600]:
me_pr['SCR 장착여부 확인 (1:장착, 2:미장착)'].value_counts(dropna=False)

1.00    3204754
2.00    1228539
NaN        2372
Name: SCR 장착여부 확인 (1:장착, 2:미장착), dtype: int64

### 추가된 정보 확인
- 컬럼별 데이터 타입 : 배기량만 int, float으로 다름.(merge는 정상적으로 작동)
- megrp1(me의 groupby), scr_ck 모두 빈 값 개수 동일
- me의 제원관리번호 중 int 타입이 존재.(1개)

In [284]:
# '배출가스인증번호', '제원관리번호', '제작사명', '연식', '차종', '엔진형식', '배기량', '엔진출력', '차명', '유럽기준', '저감장치구분', '통계자료존재유무'
# 결과
    # 배출가스인증번호 : 0
    # 제원관리번호 : 1
    # 제작사명 : 0
    # 연식 : 0
    # 차종 : 0
    # 엔진형식 : 15
        # me : 020953, scr_ck : 20953
        # 542972 : me : str, scr_ck : int
    # 엔진출력 : 1 -> 문제 없음
    # 유럽기준 : 2349 -> 문제 없음
    # 저감장치구분 : 1640030 -> 문제 없음
    # 통계자료존재유무 : 1976 -> 문제 없음
name = '통계자료존재유무'
len(set(me[name].to_list()) - set(scr_ck[name].to_list()))

1976

## 국내외 구분 빈 값 수정

In [749]:
me_pr['구분'].unique()

array(['국내', '국외', nan], dtype=object)

In [748]:
me_pr.loc[me_pr['구분'].isnull(), '제작국'].unique()

array(['한국', '독일', '스웨덴'], dtype=object)

In [750]:
me_pr.loc[(me_pr['구분'].isnull()) & (me_pr['제작국'] == '한국'), '구분'] = '국내'
me_pr.loc[(me_pr['구분'].isnull()) & (me_pr['제작국'] == '독일'), '구분'] = '국외'
me_pr.loc[(me_pr['구분'].isnull()) & (me_pr['제작국'] == '스웨덴'), '구분'] = '국외'

In [751]:
me_pr['구분'].value_counts(dropna=False)

국내    3602378
국외     833287
Name: 구분, dtype: int64

## SCR 차량과 나머지 차량 병합

In [752]:
pr.shape[0] == me_pr.shape[0] + rest.shape[0]

True

In [753]:
me_pr.shape

(4435665, 24)

In [754]:
rest.shape

(5098070, 22)

In [755]:
me_pr.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '용도', '제원관리번호', '차량말소유무', '연식', '차종',
       '차대번호', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준',
       'SCR유무', '저감장치구분', '구분', '통계자료존재유무', 'SCR 장착여부 확인 (1:장착, 2:미장착)', '확인'],
      dtype='object')

In [756]:
rest.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '용도', '제원관리번호', '차량말소유무', '연식', '차종',
       '차대번호', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준',
       'SCR유무', '저감장치구분', '구분', '통계자료존재유무'],
      dtype='object')

In [757]:
pr.shape

(9533735, 22)

In [758]:
total_pr = pd.concat([me_pr, rest], ignore_index=True)
total_pr.shape

(9533735, 24)

### [출력] 경유 차량 전체

In [759]:
today_date_dtl = today_date = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20231218_AM_113432'

In [760]:
# 1m 45.0s
total_pr.to_csv(os.path.join(raw_fold, f'전처리_경유차량전체_{today_date_dtl}.csv'), index=False)
print(os.path.join(raw_fold, f'전처리_경유차량전체_{today_date_dtl}.csv'))

D:/data/processing/urea/raw\전처리_경유차량전체_20231218_AM_113432.csv


In [761]:
total_pr = total_pr.drop('확인', axis=1)
total_pr = total_pr.rename(columns={'SCR 장착여부 확인 (1:장착, 2:미장착)':'확인'})
total_pr.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '용도', '제원관리번호', '차량말소유무', '연식', '차종',
       '차대번호', '배기량', '엔진출력', '엔진형식', '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준',
       'SCR유무', '저감장치구분', '구분', '통계자료존재유무', '확인'],
      dtype='object')

## 1\. 차종별 차량 대수

In [762]:
total_pr.shape

(9533735, 23)

In [763]:
grp1 = total_pr.groupby('차종', dropna=False)['차대번호'].count().reset_index()
grp1.loc[grp1.shape[0], ['차종', '차대번호']] = ['계', grp1['차대번호'].sum()]
grp1 = grp1.rename(columns={'차대번호':'자동차 등록대수(경유차)'})
grp1

,차종,자동차 등록대수(경유차)
0,승용,6128177.00
1,승합,608850.00
2,특수,84447.00
3,화물,2712261.00
4,계,9533735.00


In [764]:
total_pr[total_pr['확인'] == 1].shape

(3204754, 23)

In [765]:
grp2 = total_pr[total_pr['확인'] == 1].groupby('차종', dropna=False)['차대번호'].count().reset_index()
grp2.loc[grp2.shape[0], ['차종', '차대번호']] = ['계', grp2['차대번호'].sum()]
grp2 = grp2.rename(columns={'차대번호':'SCR 부착차량 대수'})
grp2

,차종,SCR 부착차량 대수
0,승용,2704075.00
1,승합,183868.00
2,특수,26894.00
3,화물,289917.00
4,계,3204754.00


In [766]:
total_pr['구분'].unique()

array(['국내', '국외', nan], dtype=object)

In [770]:
grp3 = total_pr[total_pr['확인'] == 1].groupby(['차종', '구분'], dropna=False)['차대번호'].count().unstack().reset_index()
grp3.loc[grp3.shape[0], ['차종', '국내', '국외']] = ['계', grp3['국내'].sum(), grp3['국외'].sum()]
grp3

구분,차종,국내,국외
0,승용,1976077.00,727998.00
1,승합,182953.00,915.00
2,특수,19079.00,7815.00
3,화물,229021.00,60896.00
4,계,2407130.00,797624.00


In [771]:
grp3.loc[4, ['국내', '국외']].sum()

3204754.0

## 2\. SCR 부착 화물 차량 현황

In [772]:
srt2 = total_pr[total_pr['차종'] == '화물'].reset_index(drop=True)
srt2.shape

(2712261, 23)

In [773]:
srt2['총중량'].describe()

count   2712261.00
mean       4998.62
std        6372.46
min           0.00
25%        2925.00
50%        3030.00
75%        3270.00
max      999999.00
Name: 총중량, dtype: float64

In [774]:
srt2.loc[srt2['총중량'] >= 10000, '중량구분'] = '10톤 이상(대형)'
srt2.loc[(srt2['총중량'] >= 3500) & (srt2['총중량'] < 10000), '중량구분'] = '10톤 미만(중형)'
srt2.loc[srt2['총중량'] < 3500, '중량구분'] = '3.5톤 미만(소형)'
srt2['중량구분'].value_counts(dropna=False)

3.5톤 미만(소형)    2238420
10톤 이상(대형)      265717
10톤 미만(중형)      208124
Name: 중량구분, dtype: int64

In [775]:
srt2['용도'].value_counts(dropna=False)

개인용    2378184
영업용     311152
관용       22925
Name: 용도, dtype: int64

In [776]:
srt2.loc[srt2['용도'] == '영업용', '용도구분'] = '사업용'
srt2.loc[srt2['용도'] != '영업용', '용도구분'] = '비사업용'

In [777]:
grp4 = srt2.groupby(['용도구분', '중량구분'])['차대번호'].count().reset_index()
grp4.loc[grp4.shape[0], ['용도구분', '차대번호']] = ['계', grp4['차대번호'].sum()]
grp4 = grp4.rename(columns={'차대번호':'차량대수'})
grp4

,용도구분,중량구분,차량대수
0,비사업용,10톤 미만(중형),165261.00
1,비사업용,10톤 이상(대형),111870.00
2,비사업용,3.5톤 미만(소형),2123978.00
3,사업용,10톤 미만(중형),42863.00
4,사업용,10톤 이상(대형),153847.00
5,사업용,3.5톤 미만(소형),114442.00
6,계,NaN,2712261.00


In [778]:
grp5 = srt2[srt2['확인'] == 1].groupby(['용도구분', '중량구분'])['차대번호'].count().reset_index()
grp5.loc[grp5.shape[0], ['용도구분', '차대번호']] = ['계', grp5['차대번호'].sum()]
grp5 = grp5.rename(columns={'차대번호':'차량대수'})
grp5

,용도구분,중량구분,차량대수
0,비사업용,10톤 미만(중형),11979.00
1,비사업용,10톤 이상(대형),30122.00
2,비사업용,3.5톤 미만(소형),145179.00
3,사업용,10톤 미만(중형),7813.00
4,사업용,10톤 이상(대형),81027.00
5,사업용,3.5톤 미만(소형),13797.00
6,계,NaN,289917.00


In [779]:
srt2[srt2['확인'] == 1].shape

(289917, 25)

## 3\. 제작년도 기준 SCR 부착 현황

In [780]:
srt3 = total_pr[total_pr['확인'] == 1].reset_index(drop=True)
srt3.shape

(3204754, 23)

In [781]:
grp6 = srt3.groupby(['연식', '차종'], dropna=False)['차대번호'].count().unstack().reset_index()
grp6.loc[grp6.shape[0], grp6.columns] = ['계', grp6.iloc[:, 1].sum(), grp6.iloc[:, 2].sum(), grp6.iloc[:, 3].sum(), grp6.iloc[:, 4].sum()]
grp6['계'] = grp6.iloc[:, 1:].sum(axis=1)
grp6

차종,연식,승용,승합,특수,화물,계
0,2008.00,NaN,60.00,NaN,NaN,60.00
1,2009.00,NaN,204.00,NaN,292.00,496.00
2,2010.00,NaN,67.00,3.00,781.00,851.00
3,2011.00,NaN,339.00,206.00,3546.00,4091.00
4,2012.00,1991.00,595.00,243.00,4312.00,7141.00
5,2013.00,62.00,1693.00,440.00,3369.00,5564.00
6,2014.00,8752.00,2355.00,737.00,2930.00,14774.00
7,2015.00,137268.00,16514.00,741.00,8132.00,162655.00
8,2016.00,173433.00,11544.00,540.00,6924.00,192441.00
9,2017.00,247821.00,24457.00,2665.00,18767.00,293710.00


In [782]:
grp7 = srt3.groupby(['연식', '제작사명', '차종', '차명'], dropna=False)['차대번호'].count().reset_index()
grp7 = grp7.rename(columns={'차대번호':'SCR 부착 대수'})
grp7 = grp7.sort_values(['SCR 부착 대수'], ascending=False).reset_index(drop=True)
grp7

,연식,제작사명,차종,차명,SCR 부착 대수
0,2022,현대기아,승용,포터 2.5디젤 초장축 더블캡 AT,86498
1,2020,현대기아,승용,포터 2.5디젤 초장축 더블캡 AT,81404
2,2019,현대기아,승용,카니발 2.2 디젤 9인,71017
3,2017,기아자동차(주),승용,쏘렌토,62647
4,2019,현대기아,승용,싼타페 2.0디젤 2WD,61610
...,...,...,...,...,...
19607,2020,SOFA차량,화물,포터Ⅱ (PORTERⅡ),1
19608,2020,강원도청,특수,마기루스고가사다리소방차,1
19609,2020,경북종합특장,화물,경북4.2톤암롤트럭,1
19610,2020,경북종합특장,화물,경북4X4롱바디트럭,1


### [출력] 통계

In [783]:
today_date_dot = datetime.today().strftime('%Y.%m.%d')
today_date_dot

'2023.12.18'

In [784]:
# 3.2s
with pd.ExcelWriter(os.path.join(an_fold, f'SCR부착 차량 통계({today_date_dot}).xlsx')) as writer:
    grp1.to_excel(writer, sheet_name='전체_차종', index=False)
    grp2.to_excel(writer, sheet_name='SCR_차종', index=False)
    grp3.to_excel(writer, sheet_name='SCR_국내외', index=False)
    grp4.to_excel(writer, sheet_name='화물_용도', index=False)
    grp5.to_excel(writer, sheet_name='SCR화물_용도', index=False)
    grp6.to_excel(writer, sheet_name='SCR_연식_차종', index=False)
    grp7.to_excel(writer, sheet_name='SCR_연식_제작사_차종_차명', index=False)

### [출력] SCR장착 차량

In [785]:
srt3.shape

(3204754, 23)

In [786]:
srt3.columns.to_list()

['배출가스인증번호',
 '배출가스등급',
 '최초등록일자',
 '용도',
 '제원관리번호',
 '차량말소유무',
 '연식',
 '차종',
 '차대번호',
 '배기량',
 '엔진출력',
 '엔진형식',
 '연료',
 '제작사명',
 '총중량',
 '차명',
 '제작국',
 '유럽기준',
 'SCR유무',
 '저감장치구분',
 '구분',
 '통계자료존재유무',
 '확인']

In [787]:
scr_car = srt3[[
    '차대번호',
    '배출가스인증번호',
    '제원관리번호',
    '차종',
    '용도',
    '연식',
    '구분',
    '배기량',
    '엔진출력',
    '엔진형식',
    '연료',
    '제작사명',
    '총중량',
    '차명',
    '제작국',
    
    # '배출가스등급',
    # '최초등록일자',
    # '차량말소유무',
    # '유럽기준',
    # 'SCR유무',
    # '저감장치구분',
    # '통계자료존재유무',
    # '확인'
]]
scr_car.columns

Index(['차대번호', '배출가스인증번호', '제원관리번호', '차종', '용도', '연식', '구분', '배기량', '엔진출력',
       '엔진형식', '연료', '제작사명', '총중량', '차명', '제작국'],
      dtype='object')

In [788]:
scr_ckm.loc[scr_ckm['SCR 장착여부 확인 (1:장착, 2:미장착)'] == 1, '차량대수'].sum()

3204754

In [789]:
scr_car.shape

(3204754, 15)

In [790]:
div = 50 * 10**4
for n in range(scr_car.shape[0] // div + 1):
    print(n*div, (n+1) *div)

0 500000
500000 1000000
1000000 1500000
1500000 2000000
2000000 2500000
2500000 3000000
3000000 3500000


In [791]:
today_date_dot = datetime.today().strftime('%Y.%m.%d')
today_date_dot

'2023.12.18'

In [792]:
# 20m 46.9s
div = 50 * 10**4
for n in range(scr_car.shape[0] // div + 1):
    # print(n*div, (n+1) *div)
    scr_car.iloc[n*div:(n+1)*div].to_excel(os.path.join(df_fold, f'SCR 장착 차량 샘플{n+1}({today_date_dot}).xlsx'), index=False)

### [출력] 국내외 구분 비어있는 데이터

In [744]:
scr_car[scr_car['구분'].isnull()].reset_index(drop=True).shape

(427820, 15)

In [745]:
# # 2m 58.6s
# scr_car[scr_car['구분'].isnull()].reset_index(drop=True).to_excel(os.path.join(raw_fold, f'국내외구분빈값 샘플_{today_date_dot}.xlsx'), index=False)

## 저공해조치 차량 현황

In [371]:
mgrp1 = mnt.groupby(['구분'], dropna=False)['차대번호'].count().reset_index()
mgrp1 = mgrp1.rename(columns={'차대번호':'차량대수'})
mgrp1.loc[mgrp1.shape[0], ['구분', '차량대수']] = ['합계', mgrp1['차량대수'].sum()]
mgrp1

,구분,차량대수
0,국내,2102.0
1,국외,270.0
2,합계,2372.0


In [372]:
mgrp2 = mnt.groupby(['구분', '차종'], dropna=False)['차대번호'].count().reset_index()
mgrp2 = mgrp2.rename(columns={'차대번호':'차량대수'})
mgrp2.loc[mgrp2.shape[0], ['구분', '차량대수']] = ['합계', mgrp2['차량대수'].sum()]
mgrp2

,구분,차종,차량대수
0,국내,승합,38.0
1,국내,화물,2064.0
2,국외,화물,270.0
3,합계,NaN,2372.0


In [373]:
mgrp3 = mnt.groupby(['구분', '차종유형'], dropna=False)['차대번호'].count().reset_index()
mgrp3 = mgrp3.rename(columns={'차대번호':'차량대수'})
mgrp3.loc[mgrp3.shape[0], ['구분', '차량대수']] = ['합계', mgrp3['차량대수'].sum()]
mgrp3

,구분,차종유형,차량대수
0,국내,대형,2102.0
1,국외,대형,270.0
2,합계,NaN,2372.0


In [ ]:
mgrp4 = mnt.groupby(['구분', '용도'], dropna=False)['차대번호'].count().reset_index()
mgrp4 = mgrp4.rename(columns={'차대번호':'차량대수'})
mgrp4.loc[mgrp4.shape[0], ['구분', '차량대수']] = ['합계', mgrp4['차량대수'].sum()]
mgrp4

In [374]:
mgrp5 = mnt.groupby(['구분', '차종', '차종유형', '용도'], dropna=False)['차대번호'].count().reset_index()
mgrp5 = mgrp5.rename(columns={'차대번호':'차량대수'})
mgrp5.loc[mgrp5.shape[0], ['구분', '차량대수']] = ['합계', mgrp5['차량대수'].sum()]
mgrp5

,구분,차종,차종유형,용도,차량대수
0,국내,승합,대형,개인용,34.0
1,국내,승합,대형,영업용,4.0
2,국내,화물,대형,개인용,291.0
3,국내,화물,대형,영업용,1773.0
4,국외,화물,대형,개인용,97.0
5,국외,화물,대형,영업용,173.0
6,합계,NaN,NaN,NaN,2372.0


## SCR장착 차량 현황

In [375]:
egrp1 = euro.groupby(['구분'], dropna=False)['차대번호'].count().reset_index()
egrp1 = egrp1.rename(columns={'차대번호':'차량대수'})
egrp1.loc[egrp1.shape[0], ['구분', '차량대수']] = ['합계', egrp1['차량대수'].sum()]
egrp1

,구분,차량대수
0,국내,3592589.0
1,국외,840704.0
2,합계,4433293.0


In [376]:
egrp2 = euro.groupby(['구분', '차종'], dropna=False)['차대번호'].count().reset_index()
egrp2 = egrp2.rename(columns={'차대번호':'차량대수'})
egrp2.loc[egrp2.shape[0], ['구분', '차량대수']] = ['합계', egrp2['차량대수'].sum()]
egrp2

,구분,차종,차량대수
0,국내,승용,1853816.0
1,국내,승합,198959.0
2,국내,특수,56761.0
3,국내,화물,1483053.0
4,국외,승용,763067.0
5,국외,승합,1562.0
6,국외,특수,23544.0
7,국외,화물,52531.0
8,합계,NaN,4433293.0


In [377]:
egrp3 = euro.groupby(['구분', '차종유형'], dropna=False)['차대번호'].count().reset_index()
egrp3 = egrp3.rename(columns={'차대번호':'차량대수'})
egrp3.loc[egrp3.shape[0], ['구분', '차량대수']] = ['합계', egrp3['차량대수'].sum()]
egrp3

,구분,차종유형,차량대수
0,국내,대형,1169593.0
1,국내,소형,1317655.0
2,국내,중형,1105341.0
3,국외,대형,291046.0
4,국외,소형,4637.0
5,국외,중형,545021.0
6,합계,NaN,4433293.0


In [378]:
egrp4 = euro.groupby(['구분', '용도'], dropna=False)['차대번호'].count().reset_index()
egrp4 = egrp4.rename(columns={'차대번호':'차량대수'})
egrp4.loc[egrp4.shape[0], ['구분', '차량대수']] = ['합계', egrp4['차량대수'].sum()]
egrp4

,구분,용도,차량대수
0,국내,개인용,3089301.0
1,국내,관용,41461.0
2,국내,영업용,461827.0
3,국외,개인용,765866.0
4,국외,관용,1736.0
5,국외,영업용,73102.0
6,합계,NaN,4433293.0


In [379]:
egrp5 = euro.groupby(['구분', '차종', '차종유형', '용도'], dropna=False)['차대번호'].count().reset_index()
egrp5 = egrp5.rename(columns={'차대번호':'차량대수'})
egrp5.loc[egrp5.shape[0], ['구분', '차량대수']] = ['합계', egrp5['차량대수'].sum()]
egrp5

,구분,차종,차종유형,용도,차량대수
0,국내,승용,대형,개인용,884473.0
1,국내,승용,대형,관용,2186.0
2,국내,승용,대형,영업용,150335.0
3,국내,승용,소형,개인용,1.0
4,국내,승용,중형,개인용,761883.0
...,...,...,...,...,...
60,국외,화물,소형,영업용,6.0
61,국외,화물,중형,개인용,2007.0
62,국외,화물,중형,관용,20.0
63,국외,화물,중형,영업용,2898.0


### [출력] 요소수 차량 현황

In [ ]:
today_date_dtl = today_date = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

In [ ]:
with pd.ExcelWriter(os.path.join(df_fold, f'요소수차량현황_{today_date_dtl}.xlsx')) as writer:
    mgrp1.to_excel(writer, sheet_name='저공해차량', index=False)
    mgrp2.to_excel(writer, sheet_name='저공해차량_차종', index=False)
    mgrp3.to_excel(writer, sheet_name='저공해차량_차종유형', index=False)
    mgrp4.to_excel(writer, sheet_name='저공해차량_용도', index=False)
    mgrp5.to_excel(writer, sheet_name='저공해차량_분류', index=False)
    egrp1.to_excel(writer, sheet_name='SCR장착차량', index=False)
    egrp2.to_excel(writer, sheet_name='SCR장착차량_차종', index=False)
    egrp3.to_excel(writer, sheet_name='SCR장착차량_차종유형', index=False)
    egrp4.to_excel(writer, sheet_name='SCR장착차량_용도', index=False)
    egrp5.to_excel(writer, sheet_name='SCR장착차량_분류', index=False)